In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np

<img src="https://storage.googleapis.com/lds-media/images/vgg16-architecture.width-1200.jpg"><br>
<img src="https://neurohive.io/wp-content/uploads/2018/11/vgg16-1-e1542731207177.png">

## VGG Net Using Subclassing

In [9]:
class CNNBlock1(keras.Model):
    def __init__(self,kernel_size):
        super(CNNBlock1,self).__init__()
        self.cov1 = layers.Conv2D(kernel_size,(3,3),padding="same", activation="relu")
        self.cov2 = layers.Conv2D(kernel_size,(3,3),padding="same", activation="relu")        
        self.maxpol = layers.MaxPool2D(pool_size=(2, 2),strides=(2,2))    
    def call(self,input_tensor,training = False):
        x = self.cov1(input_tensor,training = training)
        x = self.cov2(x,training = training)
        x = self.maxpol(x,training = training)
        return x
    
class CNNBlock2(keras.Model):
    def __init__(self,kernel_size):
        super(CNNBlock2,self).__init__()
        self.cov1 = layers.Conv2D(kernel_size,(3,3),padding="same", activation="relu")
        self.cov2 = layers.Conv2D(kernel_size,(3,3),padding="same", activation="relu")  
        self.cov3 = layers.Conv2D(kernel_size,(3,3),padding="same", activation="relu")        
        self.maxpol = layers.MaxPool2D(pool_size=(2, 2),strides=(2,2))    
    def call(self,input_tensor,training = False):
        x = self.cov1(input_tensor,training = training)
        x = self.cov2(x,training = training)
        x = self.cov3(x,training = training)
        x = self.maxpol(x,training = training)
        return x
    
class DenseBlock(keras.Model):
    def __init__(self,n_class):
        super(DenseBlock,self).__init__()
        self.flaten = layers.Flatten()
        self.dense1 = layers.Dense(4096,activation="relu")
        self.dense2 = layers.Dense(4096,activation="relu")        
        self.dense3 = layers.Dense(4096,activation="relu")
        self.dense4 = layers.Dense(n_class,activation="softmax")
        
    def call(self,input_tensor,training = False):
        x = self.flaten(input_tensor)
        x = self.dense1(x,training = training)
        x = self.dense2(x,training = training)
        x = self.dense3(x,training = training)
        x = self.dense4(x)
        return x
    
class VGGNet(keras.Model):
    def __init__(self):
        super(VGGNet,self).__init__()
        self.block1 = CNNBlock1(64)
        self.block2 = CNNBlock1(128)
        self.block3 = CNNBlock2(256)
        self.block4 = CNNBlock2(512)    
        self.block5 = CNNBlock2(512)
        self.dense = DenseBlock(10)
    def call(self,input_tensor,training = False):
        x = self.block1(input_tensor,training = training)
        x = self.block2(x,training = training)
        x = self.block3(x,training = training)        
        x = self.block4(x,training = training)      
        x = self.block5(x,training = training)    
        x = self.dense(x,training = training)
        return x
    
    def model(self):
        x = keras.Input(shape=(256, 256, 3) )
        output = self.call(x)
        print(output)
        return keras.Model(inputs=[x],outputs=output)

In [10]:
model = VGGNet().model()
model.summary()

In [16]:
model = keras.Sequential()
model.add(layers.Input(shape=(256, 256, 3)))
model.add(layers.Conv2D(64,(3,3),padding="same", activation="relu"))
model.add(layers.Conv2D(64,(3,3),padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2, 2),strides=(2,2)))

model.add(layers.Conv2D(128,(3,3),padding="same", activation="relu"))
model.add(layers.Conv2D(128,(3,3),padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2, 2),strides=(2,2)))

model.add(layers.Conv2D(256,(3,3),padding="same", activation="relu"))
model.add(layers.Conv2D(256,(3,3),padding="same", activation="relu"))
model.add(layers.Conv2D(256,(3,3),padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2, 2),strides=(2,2)))

model.add(layers.Conv2D(512,(3,3),padding="same", activation="relu"))
model.add(layers.Conv2D(512,(3,3),padding="same", activation="relu"))
model.add(layers.Conv2D(512,(3,3),padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2, 2),strides=(2,2)))

model.add(layers.Conv2D(512,(3,3),padding="same", activation="relu"))
model.add(layers.Conv2D(512,(3,3),padding="same", activation="relu"))
model.add(layers.Conv2D(512,(3,3),padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2, 2),strides=(2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(4096,activation="relu"))
model.add(layers.Dense(4096,activation="relu"))
model.add(layers.Dense(4096,activation="relu"))
model.add(layers.Dense(10,activation="softmax"))

In [17]:
model.summary()

In [19]:
import os
from zipfile import ZipFile
list_of_zip_file =  ['../input/dogs-vs-cats/test1.zip', '../input/dogs-vs-cats/train.zip']
for zip_file in list_of_zip_file:
    ZipFile(zip_file, mode = "r").extractall()